## ReAct

leader 负责推理，然后调用工具或结束，将「思考-行动-观察-反思」过程补充到提示语中。

In [1]:
from illufly.chat import StepByStep, ChatQwen, ChatOpenAI, ChatZhipu
from illufly.types import PromptTemplate

# leader = ChatZhipu(name="团队主管", model="glm-4-plus")
leader = ChatQwen(name="芊芊")
naming = ChatZhipu(name="起名专家", description="擅长为孩子起名，其他不会。")
writer = ChatZhipu(name="写手", description="擅长写儿歌，其他不会。")
math = ChatQwen(
    model='qwen-math-plus',
    name="数学专家",
    description="擅长解决明确的数学问题，即使是简单的数学问题也必须找我来解决。但时间安排等其他问题我不会。"
)
team = StepByStep(leader, members=[naming, writer, math])
team.provider_dict

{'members': '[{"成员名字": "起名专家", "擅长能力": "擅长为孩子起名，其他不会。"}, {"成员名字": "写手", "擅长能力": "擅长写儿歌，其他不会。"}, {"成员名字": "数学专家", "擅长能力": "擅长解决明确的数学问题，即使是简单的数学问题也必须找我来解决。但时间安排等其他问题我不会。"}]',
 'completed_teamwork': ''}

## ReWoo

In [1]:
from illufly.chat import ChatQwen
from illufly.types import PromptTemplate, BaseAgent

def get_info(name: str, age: int):
    """获得人物的特征数据"""
    return f'姓名{name}: {age}岁, 这是一个爱跳舞的女孩'

naming = ChatQwen(name="naming", description="我特别擅长起名字，每次只能起一个名字")
poem = ChatQwen(name="poem", description="我特别擅长写诗，无论是古诗还是现代诗")

# 使用上面三个工具构建 Agent
qwen = ChatQwen(memory=PromptTemplate("FLOW/ReWoo"), tools=[naming, poem, get_info])

qwen("我有5岁女儿，帮我起一个名字，然后根据她的特征数据写一首4句古诗")

Plan: 首先，使用命名工具为这个5岁的女孩起一个名字。
#E1 = naming({"prompt": "我有一个5岁的女儿，请为她起一个中文名字。"})

Plan: 接下来，使用获取信息工具来收集关于这个女孩的具体特征数据。
#E2 = get_info({"name": #E1, "age": 5})

Plan: 最后，根据所获取的女孩的名字和特征数据，利用诗歌生成工具来创作一首四句的古诗。
#E3 = poem({"prompt": "根据#E2的名字和特征，写一首四句的古诗。"})


'Plan: 首先，使用命名工具为这个5岁的女孩起一个名字。\n#E1 = naming({"prompt": "我有一个5岁的女儿，请为她起一个中文名字。"})\n\nPlan: 接下来，使用获取信息工具来收集关于这个女孩的具体特征数据。\n#E2 = get_info({"name": #E1, "age": 5})\n\nPlan: 最后，根据所获取的女孩的名字和特征数据，利用诗歌生成工具来创作一首四句的古诗。\n#E3 = poem({"prompt": "根据#E2的名字和特征，写一首四句的古诗。"})'

In [2]:
import re

text = qwen.last_output

# 正则表达式解析文本
pattern = r"Plan: (.*?)\n#(E\d+) = (\w+)[\(\[](\{.*?\})[\)\]]"
results = []

for match in re.finditer(pattern, text, re.DOTALL):
    plan_description = match.group(1).strip()
    function_name = match.group(3)
    arguments = match.group(4)  # 保留原始参数字符串，包括占位符

    result = {
        f"#{match.group(2)}": {
            "description": plan_description,
            "name": function_name,
            "arguments": arguments  # 直接存储原始参数字符串
        }
    }
    results.append(result)

results

[{'#E1': {'description': '首先，使用命名工具为这个5岁的女孩起一个名字。',
   'name': 'naming',
   'arguments': '{"prompt": "我有一个5岁的女儿，请为她起一个中文名字。"}'}},
 {'#E2': {'description': '接下来，使用获取信息工具来收集关于这个女孩的具体特征数据。',
   'name': 'get_info',
   'arguments': '{"name": #E1, "age": 5}'}},
 {'#E3': {'description': '最后，根据所获取的女孩的名字和特征数据，利用诗歌生成工具来创作一首四句的古诗。',
   'name': 'poem',
   'arguments': '{"prompt": "根据#E2的名字和特征，写一首四句的古诗。"}'}}]

In [39]:
qwen.last_output

'Plan: 首先利用naming工具为5岁的女儿起一个名字。\n#E1 = naming({"prompt": "我有一个5岁的女儿，请帮她起一个中文名字。"})\n\nPlan: 使用get_info工具获取新名字的女儿的特征数据。\n#E2 = get_info({"name": "#E1", "age": 5})\n\nPlan: 最后使用poem工具根据她的特征数据写一首四句的古诗。\n#E3 = poem({"prompt": "根据#E2的信息，写一首关于5岁的女孩的四句古诗。"})'

In [42]:
import re

regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\(([^\]]+)\)"
matches = re.findall(regex_pattern, qwen.last_output)
{"steps": matches, "plan_string": qwen.last_output}


{'steps': [('首先利用naming工具为5岁的女儿起一个名字。',
   '#E1',
   'naming',
   '{"prompt": "我有一个5岁的女儿，请帮她起一个中文名字。"})\n\nPlan: 使用get_info工具获取新名字的女儿的特征数据。\n#E2 = get_info({"name": "#E1", "age": 5})\n\nPlan: 最后使用poem工具根据她的特征数据写一首四句的古诗。\n#E3 = poem({"prompt": "根据#E2的信息，写一首关于5岁的女孩的四句古诗。"}')],
 'plan_string': 'Plan: 首先利用naming工具为5岁的女儿起一个名字。\n#E1 = naming({"prompt": "我有一个5岁的女儿，请帮她起一个中文名字。"})\n\nPlan: 使用get_info工具获取新名字的女儿的特征数据。\n#E2 = get_info({"name": "#E1", "age": 5})\n\nPlan: 最后使用poem工具根据她的特征数据写一首四句的古诗。\n#E3 = poem({"prompt": "根据#E2的信息，写一首关于5岁的女孩的四句古诗。"})'}

In [35]:

desc = "\n".join(
    [
        f'({index+1}){t.name}[{",".join([p for p in t.parameters["properties"]])}]: {t.description}。 {";".join([k+"的类型为"+v["type"]+",(目的是)"+v["description"] for k, v in t.parameters["properties"].items()])}'
        for index, t
        in enumerate(qwen.get_tools())
    ]
)
print(desc)


(1)naming[prompt]: 我特别擅长起名字，每次只能起一个名字。 prompt的类型为string,(目的是)详细描述用户问题
(2)poem[prompt]: 我特别擅长写诗，无论是古诗还是现代诗。 prompt的类型为string,(目的是)详细描述用户问题


In [6]:
qwen.provider_dict['tools_desc']

'{"type": "function", "function": {"name": "naming", "description": "我特别擅长起名字，每次只能起一个名字", "parameters": {"type": "object", "properties": {"prompt": {"type": "string", "description": "详细描述用户问题"}}, "required": ["prompt"]}}}\n{"type": "function", "function": {"name": "poem", "description": "我特别擅长写诗，无论是古诗还是现代诗", "parameters": {"type": "object", "properties": {"prompt": {"type": "string", "description": "详细描述用户问题"}}, "required": ["prompt"]}}}'

## StepByStep 智能体团队

### 构建智能体团队

In [1]:
from illufly.chat import StepByStep, ChatQwen, ChatOpenAI, ChatZhipu
from illufly.types import PromptTemplate

# leader = ChatZhipu(name="团队主管", model="glm-4-plus")
leader = ChatQwen(name="芊芊")
naming = ChatZhipu(name="起名专家", description="擅长为孩子起名，其他不会。")
writer = ChatZhipu(name="写手", description="擅长写儿歌，其他不会。")
math = ChatQwen(
    model='qwen-math-plus',
    name="数学专家",
    description="擅长解决明确的数学问题，即使是简单的数学问题也必须找我来解决。但时间安排等其他问题我不会。"
)
team = StepByStep(leader, members=[naming, writer, math])
team.provider_dict

{'members': '[{"成员名字": "起名专家", "擅长能力": "擅长为孩子起名，其他不会。"}, {"成员名字": "写手", "擅长能力": "擅长写儿歌，其他不会。"}, {"成员名字": "数学专家", "擅长能力": "擅长解决明确的数学问题，即使是简单的数学问题也必须找我来解决。但时间安排等其他问题我不会。"}]',
 'completed_teamwork': ''}

### 写儿歌

In [2]:
team("帮我写一首儿歌，关于月亮的，四句即可", verbose=False)

[AGENT] ROUND 1 @芊芊
**决策** 
当前的任务是创作一首关于月亮的儿歌，并且只需要四句。根据任务需求，我们需要的是具备创意与文字功底的工作，这正好匹配了我的团队成员中的“写手”的专长。因此，最合理的选择是将这项任务交给“写手”来完成。

**规划**
由于创作儿歌属于写手的专业范畴，无需进一步细分任务。直接将任务交予写手执行即可。

<sub_task>
{"member_name": "写手", "task_detail": "请帮我写一首关于月亮的儿歌，只需要四句。"}
</sub_task> 

至此，我们已经明确了任务并分配给了最适合的团队成员。接下来等待写手完成任务。
[AGENT] ROUND 1 @写手
月亮弯弯挂天空，
闪闪发光亮晶晶。
小星星跟着月亮走，
唱着儿歌做游戏。
[AGENT] ROUND 2 @芊芊
**观察** 根据之前团队协作记录，@写手 已经完成了编写一首关于月亮的儿歌的任务，并且儿歌满足了四句的要求。

**反思** 总任务目标是写一首关于月亮的儿歌，四句即可。@写手 提供的内容完全符合要求，因此无需进一步的修改或添加。

**决策** 当前任务已经完成，可以直接输出最终答案。

<final_answer>
月亮弯弯挂天空，
闪闪发光亮晶晶。
小星星跟着月亮走，
唱着儿歌做游戏。
</final_answer>


'\n月亮弯弯挂天空，\n闪闪发光亮晶晶。\n小星星跟着月亮走，\n唱着儿歌做游戏。\n'

In [3]:
print(team.provider_dict['completed_teamwork'])

[总任务目标] 帮我写一首儿歌，关于月亮的，四句即可
[行动] #E1 子任务交给 @写手, 要求为 请帮我写一首关于月亮的儿歌，只需要四句。
[观察] @写手 已完成 #E1 子任务，回复内容为 月亮弯弯挂天空，
闪闪发光亮晶晶。
小星星跟着月亮走，
唱着儿歌做游戏。
[最终答案] 

月亮弯弯挂天空，
闪闪发光亮晶晶。
小星星跟着月亮走，
唱着儿歌做游戏。



### 旅行建议

In [4]:
team("我想去广州玩，给我个行程概要，不要超过5句话")

[AGENT] ROUND 1 @芊芊
**决策** 
当前任务需要提供一个简短的广州旅行行程概要，这个任务并不涉及特别的专业技能，比如起名、写儿歌或者解决数学问题。因此，这个任务我可以直接完成，不需要再细分给团队成员。

**规划**
我自己来制定这个简短的行程概要。

<self_solve>
(1) 开始一天从越秀公园的早茶体验开始，享受地道的广式早餐。
(2) 接下来前往陈家祠，欣赏精美的岭南建筑艺术，并了解当地的历史文化。
(3) 下午可以在珠江新城逛逛，这里有许多购物商场和美食选择。
(4) 傍晚时分，可以去广州塔（小蛮腰）观光，俯瞰整个城市的美景。
(5) 结束一天的行程前，在北京路步行街散步购物，感受广州的夜生活。
</self_solve>

至此，我已经完成了行程概要的制定。 

<final_answer>
广州之旅建议：
1. 在越秀公园享用早茶。
2. 参观陈家祠，领略传统岭南建筑之美。
3. 在珠江新城逛街购物。
4. 晚上到广州塔观赏城市夜景。
5. 最后在北京路步行街体验广州夜生活。
</final_answer>


'\n广州之旅建议：\n1. 在越秀公园享用早茶。\n2. 参观陈家祠，领略传统岭南建筑之美。\n3. 在珠江新城逛街购物。\n4. 晚上到广州塔观赏城市夜景。\n5. 最后在北京路步行街体验广州夜生活。\n'

In [5]:
team.provider_dict

{'last_output': '\n广州之旅建议：\n1. 在越秀公园享用早茶。\n2. 参观陈家祠，领略传统岭南建筑之美。\n3. 在珠江新城逛街购物。\n4. 晚上到广州塔观赏城市夜景。\n5. 最后在北京路步行街体验广州夜生活。\n',
 'members': '[{"成员名字": "起名专家", "擅长能力": "擅长为孩子起名，其他不会。"}, {"成员名字": "写手", "擅长能力": "擅长写儿歌，其他不会。"}, {"成员名字": "数学专家", "擅长能力": "擅长解决明确的数学问题，即使是简单的数学问题也必须找我来解决。但时间安排等其他问题我不会。"}]',
 'completed_teamwork': '[总任务目标] 我想去广州玩，给我个行程概要，不要超过5句话\n[行动] 该子任务自己解决，结果为 \n(1) 开始一天从越秀公园的早茶体验开始，享受地道的广式早餐。\n(2) 接下来前往陈家祠，欣赏精美的岭南建筑艺术，并了解当地的历史文化。\n(3) 下午可以在珠江新城逛逛，这里有许多购物商场和美食选择。\n(4) 傍晚时分，可以去广州塔（小蛮腰）观光，俯瞰整个城市的美景。\n(5) 结束一天的行程前，在北京路步行街散步购物，感受广州的夜生活。\n\n[最终答案] \n\n广州之旅建议：\n1. 在越秀公园享用早茶。\n2. 参观陈家祠，领略传统岭南建筑之美。\n3. 在珠江新城逛街购物。\n4. 晚上到广州塔观赏城市夜景。\n5. 最后在北京路步行街体验广州夜生活。\n'}

### 起名字 + 写儿歌

In [6]:
team("我姓薛，请你帮我给女儿起个好名字，一个就行，再以此为题写一首4行的儿歌")

[AGENT] ROUND 1 @芊芊
**决策** 
首先，我们需要确定的是两件事情：一是为薛先生的女儿起一个好名字；二是根据这个名字创作一首四行的儿歌。考虑到“起名专家”专门负责起名，“写手”则擅长写儿歌，因此，我们应该先由起名专家来完成起名工作，之后再由写手根据所起的名字来编写儿歌。

**行动**
<sub_task>
{"member_name": "起名专家", "task_detail": "薛先生希望给他的女儿起一个有意义且美好的名字，请根据薛姓的特点，结合中华文化的内涵，为薛先生的女儿起一个名字。"}
</sub_task> 

等待起名专家给出名字后，下一步行动将会是让写手根据这个名字创作儿歌。
[AGENT] ROUND 1 @起名专家
薛先生您好，结合薛姓的特点和中华文化的内涵，我为您的女儿推荐以下名字：

薛雅心

“雅”字寓意高雅、优雅，符合中华文化中对女性美好的期待；“心”字则代表内心、情感，寓意女儿心地善良，拥有纯真的心灵。这个名字既体现了薛姓的韵味，又富有文化内涵和美好寓意。希望您会喜欢！
[AGENT] ROUND 2 @芊芊
**观察**

从当前的任务需求来看，我们已经完成了为薛先生的女儿起名的部分，名字为“薛雅心”。接下来需要以这个名字为主题创作一首四行的儿歌。

**反思**

既然名字已经确定，并且创作儿歌属于“写手”的专长，因此应该将接下来的任务交给“写手”。

**决策**

下一步的任务是创作儿歌，必须由“写手”来完成。

**行动**

<sub_task>
{"member_name": "写手", "task_detail": "以薛雅心的名字为主题，创作一首四行的儿歌。这首儿歌应该充满童趣，同时也能体现薛雅心这个名字的美好寓意。"}
</sub_task>
[AGENT] ROUND 2 @写手
薛薛小雅心，阳光灿烂笑，
雅韵传千里，心灵美如花。
心系快乐源，成长步步高，
幸福歌声里，薛雅心儿谣。
[AGENT] ROUND 3 @芊芊
**观察** 根据之前的团队协作记录，我们的团队已经完成了两个关键子任务：为薛先生的女儿起了一个名字“薛雅心”，并且以这个名字为主题创作了一首四行的儿歌。接下来我们需要确认这两项成果是否满足了总任务目标。

**反思** 起名专家提供的名字“薛雅心”富有文化内涵和美好的寓意

'\n(最终答案如下)\n\n为薛先生的女儿起的名字是：“薛雅心”。\n\n以这个名字为主题的儿歌是：\n薛薛小雅心，阳光灿烂笑，\n雅韵传千里，心灵美如花。\n心系快乐源，成长步步高，\n幸福歌声里，薛雅心儿谣。\n\n'

In [7]:
writer.memory

[{'role': 'user',
  'content': '以薛雅心的名字为主题，创作一首四行的儿歌。这首儿歌应该充满童趣，同时也能体现薛雅心这个名字的美好寓意。'},
 {'role': 'assistant',
  'content': '薛薛小雅心，阳光灿烂笑，\n雅韵传千里，心灵美如花。\n心系快乐源，成长步步高，\n幸福歌声里，薛雅心儿谣。'}]

In [8]:
print(team.provider_dict['members'])

[{"成员名字": "起名专家", "擅长能力": "擅长为孩子起名，其他不会。"}, {"成员名字": "写手", "擅长能力": "擅长写儿歌，其他不会。"}, {"成员名字": "数学专家", "擅长能力": "擅长解决明确的数学问题，即使是简单的数学问题也必须找我来解决。但时间安排等其他问题我不会。"}]


In [9]:
print(team.provider_dict['completed_teamwork'])

[总任务目标] 我姓薛，请你帮我给女儿起个好名字，一个就行，再以此为题写一首4行的儿歌
[行动] #E1 子任务交给 @起名专家, 要求为 为薛姓的女儿起一个寓意美好且独特的好名字。
[观察] @起名专家 已完成 #E1 子任务，回复内容为 薛雅涵
寓意：雅涵表示文雅内涵，希望女儿拥有高雅的气质和丰富的内涵，寓意美好且独特。
[行动] #E1 子任务交给 @写手, 要求为 以薛雅涵为题，创作一首四行的儿歌。
[观察] @写手 已完成 #E1 子任务，回复内容为 薛雅涵，笑颜开，
歌声飘，童趣在。
阳光里，快乐跑，
愿你成长，如花盛开。
[最终答案] 

最终答案：女儿的名字为“薛雅涵”。以这个名字为题的儿歌如下：
薛雅涵，笑颜开，
歌声飘，童趣在。
阳光里，快乐跑，
愿你成长，如花盛开。



### 数学问题

In [9]:
team("119乘以321是多少?")

[AGENT] ROUND 1 @芊芊
**决策** 
当前任务是计算119乘以321的结果。这是一个明确的数学问题，直接计算即可得出答案。

根据团队成员的能力，这个问题应该交给“数学专家”来解决，因为他擅长解决明确的数学问题。

**行动**
<sub_task>
{"member_name": "数学专家", "task_detail": "计算119乘以321的结果。"}
</sub_task> 

既然已经分配了任务给“数学专家”，接下来等待他的计算结果。
[AGENT] ROUND 1 @数学专家
要计算 \(119 \times 321\) 的结果，我们可以使用乘法的分配律（对于二项式，也称为FOIL方法）。首先，我们将每个数表示为100的和或差：

\[119 = 100 + 19\]
\[321 = 300 + 21\]

现在，我们使用分配律来展开乘积 \((100 + 19)(300 + 21)\)：

\[
(100 + 19)(300 + 21) = 100 \cdot 300 + 100 \cdot 21 + 19 \cdot 300 + 19 \cdot 21
\]

接下来，我们分别计算每个项：

\[
100 \cdot 300 = 30000
\]
\[
100 \cdot 21 = 2100
\]
\[
19 \cdot 300 = 5700
\]
\[
19 \cdot 21 = 399
\]

现在，我们将这些乘积相加：

\[
30000 + 2100 + 5700 + 399
\]

首先，将30000和2100相加：

\[
30000 + 2100 = 32100
\]

接下来，将5700加到结果上：

\[
32100 + 5700 = 37800
\]

最后，将399加到结果上：

\[
37800 + 399 = 38199
\]

因此，\(119 \times 321\) 的结果是 \(\boxed{38199}\)。
[AGENT] ROUND 2 @芊芊
<final_answer>
根据 @数学专家 的计算，119乘以321的结果是38199。
</final_answer> 

鉴于此任务已经由我们的数学专家计算完毕并且得出了最终的答案，目前不需要进一步的任务规划或子任务分配。任务可以到此为止。

'\n根据 @数学专家 的计算，119乘以321的结果是38199。\n'

In [13]:
leader("119乘以321是多少", new_chat=True)

119乘以321的结果是 38299。


'119乘以321的结果是 38299。'

In [14]:
team.provider_dict

{'last_output': '\n根据 @数学专家 的计算，119乘以321的结果是38199。\n',
 'members': '[{"成员名字": "起名专家", "擅长能力": "擅长为孩子起名，其他不会。"}, {"成员名字": "写手", "擅长能力": "擅长写儿歌，其他不会。"}, {"成员名字": "数学专家", "擅长能力": "擅长解决明确的数学问题，即使是简单的数学问题也必须找我来解决。但时间安排等其他问题我不会。"}]',
 'completed_teamwork': '[总任务目标] 119乘以321是多少?\n[行动] #E1 子任务交给 @数学专家, 要求为 计算119乘以321的结果。\n[观察] @数学专家 已完成 #E1 子任务，回复内容为 要计算 \\(119 \\times 321\\) 的结果，我们可以使用乘法的分配律（对于二项式，也称为FOIL方法）。首先，我们将每个数表示为100的和或差：\n\n\\[119 = 100 + 19\\]\n\\[321 = 300 + 21\\]\n\n现在，我们使用分配律来展开乘积 \\((100 + 19)(300 + 21)\\)：\n\n\\[\n(100 + 19)(300 + 21) = 100 \\cdot 300 + 100 \\cdot 21 + 19 \\cdot 300 + 19 \\cdot 21\n\\]\n\n接下来，我们分别计算每个项：\n\n\\[\n100 \\cdot 300 = 30000\n\\]\n\\[\n100 \\cdot 21 = 2100\n\\]\n\\[\n19 \\cdot 300 = 5700\n\\]\n\\[\n19 \\cdot 21 = 399\n\\]\n\n现在，我们将这些乘积相加：\n\n\\[\n30000 + 2100 + 5700 + 399\n\\]\n\n首先，将30000和2100相加：\n\n\\[\n30000 + 2100 = 32100\n\\]\n\n接下来，将5700加到结果上：\n\n\\[\n32100 + 5700 = 37800\n\\]\n\n最后，将399加到结果上：\n\n\\[\n37800 + 399 = 38199\n\\]\n\n因此，\\(119 \\times 

### 较难的数学问题

In [2]:
question = """
一种减速机的价格是750元，一家企业需要购买12台。
每一台减速机运行一小时的电费是0.5元，企业每天运行这些减速机8小时，单周休1天，双周休六日两天。
请计算企业购买这些设备，并运行4周，总花费多少？
"""

In [4]:
team(question)

[AGENT] ROUND 1 @芊芊
**思考**

首先，我们需要确定企业购买这些减速机的总成本，然后计算出在四周内运行这些机器所产生的电费。最后，将两者相加得到总花费。

具体来说，我们有以下信息：

1. 每台减速机的价格为750元，企业需要购买12台。
2. 每台减速机运行一小时的电费是0.5元。
3. 企业每天运行这些减速机8小时。
4. 单周休1天，双周休6天，这意味着在四个星期内，会有两周是工作6天，另外两周是工作5天。

接下来，我们将根据以上信息进行计算。

**决策**
由于这些任务不涉及任何团队成员的专业领域，我可以自行解决。

<self_solve>
1. 计算购买减速机的总成本：750元/台 * 12台 = 9000元
2. 计算四周内总共工作的天数：对于单周休一天的情况，每周工作6天；对于双周休两天的情况，每周工作5天。所以，在四周内，工作天数为2*6 + 2*5 = 22天。
3. 计算每天的电费：每台机器每天运行8小时，因此每天的电费为0.5元/小时 * 8小时 = 4元/台。那么，所有机器一起每天的电费为4元/台 * 12台 = 48元。
4. 计算四周内的总电费：48元/天 * 22天 = 1056元。
5. 最后，计算总花费：购买成本+电费=9000元 + 1056元 = 10056元。
</self_solve>

<final_answer>
企业购买这些设备，并运行4周，总花费为10056元。
</final_answer>


'\n企业购买这些设备，并运行4周，总花费为10056元。\n'

In [5]:
print(team.provider_dict['completed_teamwork'])

[总任务目标] 
一种减速机的价格是750元，一家企业需要购买12台。
每一台减速机运行一小时的电费是0.5元，企业每天运行这些减速机8小时，单周休1天，双周休六日两天。
请计算企业购买这些设备，并运行4周，总花费多少？

[行动] 该子任务自己解决，结果为 
1. 计算购买减速机的总成本：750元/台 * 12台 = 9000元
2. 计算四周内总共工作的天数：对于单周休一天的情况，每周工作6天；对于双周休两天的情况，每周工作5天。所以，在四周内，工作天数为2*6 + 2*5 = 22天。
3. 计算每天的电费：每台机器每天运行8小时，因此每天的电费为0.5元/小时 * 8小时 = 4元/台。那么，所有机器一起每天的电费为4元/台 * 12台 = 48元。
4. 计算四周内的总电费：48元/天 * 22天 = 1056元。
5. 最后，计算总花费：购买成本+电费=9000元 + 1056元 = 10056元。

[最终答案] 

企业购买这些设备，并运行4周，总花费为10056元。

